In [8]:
%matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url= 'https://raw.githubusercontent.com/BigDataGal/Python-for-Data-Science/master/titanic-train.csv'
titantic=pd.read_csv(url)

titantic.columns=["PassengerId","Survived","Pclass","Name","Sex","Age","SibSp","Parch","Ticket","Fare","Cabin","Embarked"]
titantic.head()

Using matplotlib backend: Qt5Agg


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [61]:
titantic.isnull().sum()
#可见age，与cabin，embarked是存在缺失的

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# titantic.info()

In [83]:
titantic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


# 清洗数据

In [84]:
titantic_data=titantic.drop(["Name","PassengerId","Cabin","Ticket"],1)
#1删除列，0删除行


In [85]:
def age_approx(cols):
    Age=cols[0]
    Pclass=cols[1]
    if pd.isnull(Age):
        if Pclass==1:
            return 37
        elif Pclass==2:
            return 29
        else:
            return 24
    else:
        return Age
    
titantic_data["Age"]=titantic_data[["Age","Pclass"]].apply(age_approx,axis=1)
titantic_data.dropna(inplace=True)
titantic_data.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

#### gender=pd.get_dummies(titantic_data['Sex'],drop_first=True)
embark_location=pd.get_dummies(titantic_data["Embarked"],drop_first=True)
titantic_data.drop(["Sex","Embarked"],axis=1,inplace=True)
titantic_reg=pd.concat([titantic_data,gender,embark_location],axis=1)
titantic_reg.head()

In [86]:
gender=pd.get_dummies(titantic_data['Sex'],drop_first=True)
embark_location=pd.get_dummies(titantic_data["Embarked"],drop_first=True) 
titantic_data.drop(["Sex","Embarked"],axis=1,inplace=True) 
titantic_reg=pd.concat([titantic_data,gender,embark_location],axis=1) 
titantic_reg.head()

,Survived,Pclass,Age,SibSp,Parch,Fare,male,Q,S
0,0,3,22.0,1,0,7.2500,1,0,1
1,1,1,38.0,1,0,71.2833,0,0,0
2,1,3,26.0,0,0,7.9250,0,0,1
3,1,1,35.0,1,0,53.1000,0,0,1
4,0,3,35.0,0,0,8.0500,1,0,1


In [87]:
import seaborn as sb
sb.heatmap(titantic_reg.corr())


In [88]:
titantic_reg.drop(["Fare","Pclass"],axis=1,inplace=True)
titantic_reg.head()

,Survived,Age,SibSp,Parch,male,Q,S
0,0,22.0,1,0,1,0,1
1,1,38.0,1,0,0,0,0
2,1,26.0,0,0,0,0,1
3,1,35.0,1,0,0,0,1
4,0,35.0,0,0,1,0,1


# 清洗完毕

In [97]:
y=titantic_reg.iloc[:,0].values
x=titantic_reg.iloc[:,1:6].values
#from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.4,random_state=25)

In [100]:
from sklearn.linear_model import LogisticRegression
logit_model=LogisticRegression()
logit_model.fit(x_train,y_train)


C:\Users\Double2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [102]:
y_pred=logit_model.predict(x_test)

In [106]:
pd.crosstab(y_test,y_pred,rownames=["True"],colnames=["Predicted"],margins=True)

Predicted,0,1,All
True,,,
0,182,30,212
1,46,98,144
All,228,128,356


In [107]:
from sklearn.metrics import confusion_matrix
confusion_matrix=confusion_matrix(y_test,y_pred)
confusion_matrix

array([[182,  30],
       [ 46,  98]], dtype=int64)